In [1]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
import warnings
warnings.filterwarnings('ignore')
from empyrical import max_drawdown,cum_returns_final,cum_returns,sortino_ratio,sharpe_ratio,calmar_ratio,annual_return,annual_volatility

import matplotlib.pyplot as plt

import time
import datetime as dt
from datetime import timedelta
from pytz import timezone

import yfinance as yf
from datetime import date
import itertools as it

In [2]:
from platform import python_version

print(python_version())

3.7.4


# PARAMETERS

In [3]:
Ex_MA = 5
look_back_window = 20 #zscores

###EMA
long_mvg = 5
short_mvg = 2
shift_close_open = 'yes' #if EMA based on open and we have open as rebalance, if shift_close_open='yes' the signal iis shifted one day
EMA_zscore       = 'yes' #cross-sectional zscore (mainly merging purpose) 



# LOAD DATA

In [4]:
import_path = '/Users/niclaswolnerhanssen/Desktop/LINC/Sentiment LYNX/1_DATA_MINING/'

BLOOMBERG_TOT = pd.read_excel(
    import_path+'BLOOMBERG_TOT.xlsx',index_col=0)
BLOOMBERG_POS = pd.read_excel(
    import_path+'BLOOMBERG_POS.xlsx',index_col=0)

REDDIT_TOT = pd.read_excel(
    import_path+'REDDIT_TOT.xlsx',index_col=0)
REDDIT_POS = pd.read_excel(
    import_path+'REDDIT_POS.xlsx',index_col=0)

TWITTER_TOT = pd.read_excel(
    import_path+'TWITTER_TOT.xlsx',index_col=0)
TWITTER_POS = pd.read_excel(
    import_path+'TWITTER_POS.xlsx',index_col=0)

XLK_OPEN_ETF = pd.read_excel(
    import_path+'XLK_OPEN_ETF.xlsx',index_col=0)
XLK_CLOSE_ETF = pd.read_excel(
    import_path+'XLK_CLOSE_ETF.xlsx',index_col=0)

STOCK_PRICES_CLOSE = pd.read_excel(
    import_path+'STOCK_PRICES_CLOSE.xlsx',index_col=0)
STOCK_PRICES_OPEN = pd.read_excel(
    import_path+'STOCK_PRICES_OPEN.xlsx',index_col=0)

# DATA PRE-PROCESSING

In [5]:
BLOOMBERG_TOT_EXP = BLOOMBERG_TOT.ewm(span=Ex_MA).mean(skipna=True)
BLOOMBERG_POS_EXP = BLOOMBERG_POS.ewm(span=Ex_MA).mean(skipna=True)
#Positve in relation to the total:
BLOOMBERG     = BLOOMBERG_POS_EXP.div(BLOOMBERG_TOT_EXP)
print(BLOOMBERG.shape) 
print('START: '+str(BLOOMBERG.index[0]))
print('END: '+str(BLOOMBERG.index[-1]))

(245, 75)
START: 2020-11-25 00:00:00
END: 2021-11-02 00:00:00


In [6]:
REDDIT_TOT_EXP = REDDIT_TOT.ewm(span=Ex_MA).mean(skipna=True)
REDDIT_POS_EXP = REDDIT_POS.ewm(span=Ex_MA).mean(skipna=True)
#Positve in relation to the total:
REDDIT     = REDDIT_POS_EXP.div(REDDIT_TOT_EXP)
REDDIT     = REDDIT[REDDIT.index[0]:BLOOMBERG.index[-1]]

print(REDDIT.shape) 
print('START: '+str(REDDIT.index[0]))
print('END: '+str(REDDIT.index[-1]))

(238, 75)
START: 2020-11-25 00:00:00
END: 2021-11-02 00:00:00


In [7]:
TWITTER_TOT_EXP = TWITTER_TOT.ewm(span=Ex_MA).mean(skipna=True)
TWITTER_POS_EXP = TWITTER_POS.ewm(span=Ex_MA).mean(skipna=True)
#Positve in relation to the total:
TWITTER     = TWITTER_POS_EXP.div(TWITTER_TOT_EXP)
TWITTER     = TWITTER[TWITTER.index[0]:BLOOMBERG.index[-1]]

print(TWITTER.shape) 
print('START: '+str(TWITTER.index[0]))
print('END: '+str(TWITTER.index[-1]))

(224, 75)
START: 2020-11-25 00:00:00
END: 2021-11-02 00:00:00


# ZSCORE TRANSFORMATION

### V2: Cross-sectional first, then historical zscores

In [8]:
def zscore_columns(data_df,look_back_window = 30,skipna=True):
    df = data_df.copy()
    for i in df.columns:
        df[i] = (df[i]-df[i].rolling(look_back_window).mean(skipna=skipna))/df[i].rolling(look_back_window).std(skipna=skipna)
                
    df = df[look_back_window:]
                
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
        
    return df



In [9]:
#stock relative
def zscore_rows(data_df,skipna=True):
    df                       = data_df.copy()
    df_mean                  = df.mean(axis=1,skipna=skipna)
    df_std                   = df.std(axis=1,skipna=skipna)
    
    for i in df.columns:
        df[i] = (df[i]-df_mean)/df_std
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
        
    return df

In [10]:
def intersected_index_dfs(df_1,df_2):
    #intersected_index is a new index with elements common to both df_1.index & df_2.index
    intersected_index = df_1.index.intersection(df_2.index, sort=False)
    #filter the dfs with the new, intersected, index
    df_1 = df_1[df_1.index.isin(intersected_index)]
    df_2 = df_2[df_2.index.isin(intersected_index)]
           
    return df_1, df_2

In [11]:
def merge_zscores(df1,df2):
    #replace nans with zeros
    df1 = df1.replace(np.nan,0)
    df2 = df2.replace(np.nan,0)
    
    df1, df2 = intersected_index_dfs(df1,df2)

    if (df1.index != df2.index).any():
        raise Exception('Index does not match')
    
    if (df1.columns != df2.columns).any():
        raise Exception('Columns does not match')
        
    #simple addition between dfs    
    df_merged = df1.add(df2)
    #replace zeros with nans 
    df_merged = df_merged.replace(0,np.nan)
    
    return df_merged
    

In [12]:
V2_TWITTER_ZSCORED         = zscore_rows(TWITTER,skipna=True)
V2_REDDIT_ZSCORED          = zscore_rows(REDDIT ,skipna=True)
V2_BLOOMBERG_ZSCORED       = zscore_rows(BLOOMBERG,skipna=True)

V2_TWITTER_ZSCORED         = zscore_columns(V2_TWITTER_ZSCORED  ,look_back_window = look_back_window,skipna=True)
V2_REDDIT_ZSCORED          = zscore_columns(V2_REDDIT_ZSCORED   ,look_back_window = look_back_window,skipna=True)
V2_BLOOMBERG_ZSCORED       = zscore_columns(V2_BLOOMBERG_ZSCORED,look_back_window = look_back_window,skipna=True)

# Fix for trading days

In [13]:
for i in [V2_TWITTER_ZSCORED ,V2_REDDIT_ZSCORED,V2_BLOOMBERG_ZSCORED]:
    print(i.shape)
    print('START: '+str(i.index[0]))
    print('END: '+str(i.index[-1]))

(204, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(218, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(225, 75)
START: 2020-12-23 00:00:00
END: 2021-11-02 00:00:00


In [14]:
#fix bloomberg starting date
V2_BLOOMBERG_ZSCORED = V2_BLOOMBERG_ZSCORED[V2_TWITTER_ZSCORED.index[0]:V2_BLOOMBERG_ZSCORED.index[-1]]

In [15]:
def filter_df_for_trading_days(df):
    df_1 = df.copy()
    nyse = mcal.get_calendar('NYSE')
    trading_days_index = nyse.valid_days(start_date=df_1.index[0], end_date=df_1.index[-1])
    trading_days_index = trading_days_index.tz_localize(tz=None)
    #filter df_1 for trading_days_index
    try:
        df_1 = df_1.loc[trading_days_index]
    except KeyError:
        pandas = pd.DataFrame(index=trading_days_index)
        pandas['DUMMY'] = np.ones(len(pandas))
        merged = pd.concat([pandas,df_1],axis=1)#trading days will result in nans for df_1 data
        merged = merged.drop('DUMMY',axis=1)    #remove dummy-column
        merged = merged.loc[trading_days_index] #filter for trading days only and remove everything else
        df_1   = merged
           
    return df_1

In [16]:
V2_BLOOMBERG_ZSCORED  = filter_df_for_trading_days(V2_BLOOMBERG_ZSCORED)
V2_REDDIT_ZSCORED     = filter_df_for_trading_days(V2_REDDIT_ZSCORED)
V2_TWITTER_ZSCORED    = filter_df_for_trading_days(V2_TWITTER_ZSCORED)


In [17]:
for i in [V2_TWITTER_ZSCORED ,V2_REDDIT_ZSCORED,V2_BLOOMBERG_ZSCORED]:
    print(i.shape)
    print('START: '+str(i.index[0]))
    print('END:   '+str(i.index[-1]))

(216, 75)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00


### Correlation between V2 zscores:

In [18]:
twitt, redd  = intersected_index_dfs(V2_TWITTER_ZSCORED,V2_REDDIT_ZSCORED)
twitt, bloom = intersected_index_dfs(twitt,V2_BLOOMBERG_ZSCORED)
twitt, redd  = intersected_index_dfs(twitt,redd)

corr_df = pd.concat([twitt.mean(axis=1,skipna=True),redd.mean(axis=1,skipna=True),bloom.mean(axis=1,skipna=True)],axis=1)
corr_df.columns = ['TWITTER V2','REDDIT V2','BLOOMBERG V2']
corr_df.corr()



,TWITTER V2,REDDIT V2,BLOOMBERG V2
TWITTER V2,1.000000,0.120829,0.112249
REDDIT V2,0.120829,1.000000,0.178436
BLOOMBERG V2,0.112249,0.178436,1.000000


### Merge

In [19]:
V2_TWITTER_REDDIT_ZSCORED           = merge_zscores(V2_TWITTER_ZSCORED,V2_REDDIT_ZSCORED)
V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED = merge_zscores(V2_TWITTER_REDDIT_ZSCORED,V2_BLOOMBERG_ZSCORED)

# Exponential Moving Average

In [20]:
#Momentum EMA Cross-over %-diff
def EMA_percentage_diff(    close_price, 
                            long_mvg, 
                            short_mvg,
                            shift_close_open       = 'yes',
                            EMA_zscore             = 'yes',
                  
                             ):
    
    MA_df = pd.DataFrame()
    
    for i in close_price.columns:
        df = close_price[[i]]
        df['EMA_long']  = df[i].ewm(span=long_mvg).mean(skipna=True)
        df['EMA_short'] = df[i].ewm(span=short_mvg).mean(skipna=True)
        df['EMA %'] = (df['EMA_short']-df['EMA_long'])/df['EMA_long']
            
        MA_df[i] = df['EMA %'].iloc[long_mvg:]
    
    if EMA_zscore =='yes':
        MA_df = zscore_rows(MA_df,skipna=True)            
        
    ##if the signal is based on close price and we trade at open: 
    ##shift the signal on day forward to line up with open.
    if shift_close_open == 'yes':
        MA_df = MA_df.shift()
        MA_df = MA_df.iloc[1:,]  
    
    return MA_df

In [21]:
V2_EMA_ZSCORED = EMA_percentage_diff(    
                                      close_price       = STOCK_PRICES_CLOSE, 
                                      long_mvg          = long_mvg, 
                                      short_mvg         = short_mvg,
                                      shift_close_open  = shift_close_open,
                                      EMA_zscore        = EMA_zscore,
                  
                             )

V2_EMA_ZSCORED = V2_EMA_ZSCORED[V2_TWITTER_ZSCORED.index[0]:V2_BLOOMBERG_ZSCORED.index[-1]]
V2_EMA_ZSCORED = filter_df_for_trading_days(V2_EMA_ZSCORED)

In [22]:
for i in [V2_EMA_ZSCORED]:
    print(i.shape)
    print('START: '+str(i.index[0]))
    print('END:   '+str(i.index[-1]))

(216, 75)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00


In [23]:
V2_EMA_TWITTER_ZSCORED                  = merge_zscores(V2_TWITTER_ZSCORED                 , V2_EMA_ZSCORED)
V2_EMA_REDDIT_ZSCORED                   = merge_zscores(V2_REDDIT_ZSCORED                  , V2_EMA_ZSCORED)
V2_EMA_BLOOMBERG_ZSCORED                = merge_zscores(V2_BLOOMBERG_ZSCORED               , V2_EMA_ZSCORED)
V2_EMA_TWITTER_REDDIT_ZSCORED           = merge_zscores(V2_TWITTER_REDDIT_ZSCORED          , V2_EMA_ZSCORED)
V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED = merge_zscores(V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED, V2_EMA_ZSCORED) 



In [24]:
feature_df_list = [
                    V2_TWITTER_ZSCORED,
                    V2_REDDIT_ZSCORED,
                    V2_BLOOMBERG_ZSCORED,
                    V2_TWITTER_REDDIT_ZSCORED,
                    V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED,
                    V2_EMA_ZSCORED,
                    V2_EMA_TWITTER_ZSCORED,
                    V2_EMA_REDDIT_ZSCORED,
                    V2_EMA_BLOOMBERG_ZSCORED,
                    V2_EMA_TWITTER_REDDIT_ZSCORED,
                    V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED
    
                     ]

In [25]:
for i in feature_df_list:
    print(i.shape)
    print('START: '+str(i.index[0]))
    print('END: '+str(i.index[-1]))

(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00
(216, 75)
START: 2020-12-24 00:00:00
END: 2021-11-02 00:00:00


# Returns - Open to Close

In [26]:
#######Transpos columns df
def mkt_neu_returns(  
                   open_price,
                   close_price,
                   open_to_close       = 'no',
                   open_to_open        = 'no',
                   equal_index_ret     = 'yes',
                   fwd_ret             = 'yes',

                ):
    
    if len(open_price) != len(close_price):
        raise Exception('close and open price have different lengths')
    
    if open_to_close == 'yes':
        data = (close_price/open_price) -1
        returns = data.copy()
        
    if open_to_open == 'yes':
        data = (open_price/open_price.shift())-1
        data = data.shift(-1)
        data = data.dropna()
        returns = data.copy()
            
    if equal_index_ret == 'yes':
        mean_ret = data.mean(axis=1,skipna=True)
        data = pd.DataFrame(mean_ret)
        data.columns = ['EQUAL INDEX']
        #for i in data.columns:
            #data[i] = mean_ret 
                
    if fwd_ret == 'yes':
        data = returns    
        
    return data

In [27]:
V2_OPEN_TO_CLOSE_STOCK_RETURNS =  mkt_neu_returns(
                                                open_price      = STOCK_PRICES_OPEN,
                                                close_price     = STOCK_PRICES_CLOSE,
                                                open_to_close   = 'yes',
                                                open_to_open    = 'no',
                                                equal_index_ret = 'no',
                                                fwd_ret         = 'yes'
    
                                               )


V2_OPEN_TO_CLOSE_STOCK_RETURNS = V2_OPEN_TO_CLOSE_STOCK_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]

In [28]:
V2_OPEN_TO_CLOSE_EQUALW_RETURNS =   mkt_neu_returns(
                                                open_price      = STOCK_PRICES_OPEN,
                                                close_price     = STOCK_PRICES_CLOSE,
                                                open_to_close   = 'yes',
                                                open_to_open    = 'no',
                                                equal_index_ret = 'yes',
                                                fwd_ret         = 'no'
    
                                              )

V2_OPEN_TO_CLOSE_EQUALW_RETURNS = V2_OPEN_TO_CLOSE_EQUALW_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]

In [29]:
V2_OPEN_TO_CLOSE_XLK_RETURNS =   mkt_neu_returns(
                                                open_price      = XLK_OPEN_ETF,
                                                close_price     = XLK_CLOSE_ETF,
                                                open_to_close   = 'yes',
                                                open_to_open    = 'no',
                                                equal_index_ret = 'no',
                                                fwd_ret         = 'yes'
    
                                              )

V2_OPEN_TO_CLOSE_XLK_RETURNS = V2_OPEN_TO_CLOSE_XLK_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]


In [30]:
for i in [V2_OPEN_TO_CLOSE_STOCK_RETURNS,V2_OPEN_TO_CLOSE_EQUALW_RETURNS,V2_OPEN_TO_CLOSE_XLK_RETURNS]:
    print(i.shape)
    print('START: '+str(i.index[0]))
    print('END:   '+str(i.index[-1]))

(216, 75)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00
(216, 1)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00
(216, 1)
START: 2020-12-24 00:00:00
END:   2021-11-02 00:00:00


# Returns - Open to Open

In [31]:
V2_OPEN_TO_OPEN_STOCK_RETURNS =  mkt_neu_returns(
                                                open_price      = STOCK_PRICES_OPEN,
                                                close_price     = STOCK_PRICES_OPEN,
                                                open_to_close   = 'no',
                                                open_to_open    = 'yes',
                                                equal_index_ret = 'no',
                                                fwd_ret         = 'yes'
    
                                               )


V2_OPEN_TO_OPEN_STOCK_RETURNS = V2_OPEN_TO_OPEN_STOCK_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]

In [32]:
V2_OPEN_TO_OPEN_EQUALW_RETURNS =   mkt_neu_returns(
                                                open_price      = STOCK_PRICES_OPEN,
                                                close_price     = STOCK_PRICES_OPEN,
                                                open_to_close   = 'no',
                                                open_to_open    = 'yes',
                                                equal_index_ret = 'yes',
                                                fwd_ret         = 'no'
    
                                              )

V2_OPEN_TO_OPEN_EQUALW_RETURNS = V2_OPEN_TO_OPEN_EQUALW_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]

In [33]:
V2_OPEN_TO_OPEN_XLK_RETURNS =   mkt_neu_returns(
                                                open_price      = XLK_OPEN_ETF,
                                                close_price     = XLK_OPEN_ETF,
                                                open_to_close   = 'no',
                                                open_to_open    = 'yes',
                                                equal_index_ret = 'no',
                                                fwd_ret         = 'yes'
    
                                              )

V2_OPEN_TO_OPEN_XLK_RETURNS = V2_OPEN_TO_OPEN_XLK_RETURNS[V2_TWITTER_ZSCORED.index[0]:V2_TWITTER_ZSCORED.index[-1]]

# Transpose all dfs

In [34]:
def transpose(df,feature_name = 'REDDIT_EGEN_TOT_SENTIMENT_COUNT'):
    value_vars  = df.columns
    df.index.name = 'Date'
    df          = df.reset_index()
        #Transpose the DataFrame: on column, stocks stacked rowwise with dates:
    df = pd.melt(df,
                    id_vars    = ['Date'],
                    value_vars = value_vars,
                    var_name   ='Stock',
                    value_name = feature_name)

    df = df.sort_values(['Date','Stock'])
    df = df.reset_index()
    df = df.drop('index',axis=1)
    
    return df

In [35]:
V2_TWITTER_ZSCORED                      = transpose(V2_TWITTER_ZSCORED,                     feature_name  = 'TWITTER')              
V2_REDDIT_ZSCORED                       = transpose(V2_REDDIT_ZSCORED,                      feature_name  = 'REDDIT')
V2_BLOOMBERG_ZSCORED                    = transpose(V2_BLOOMBERG_ZSCORED,                   feature_name  = 'BLOOMBERG')
V2_TWITTER_REDDIT_ZSCORED               = transpose(V2_TWITTER_REDDIT_ZSCORED,              feature_name  = 'TWITTER_REDDIT')
V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED     = transpose(V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED,    feature_name  = 'TWITTER_REDDIT_BLOOMBERG')
V2_EMA_ZSCORED                          = transpose(V2_EMA_ZSCORED,                         feature_name  = 'EMA_ZSCORED')
V2_EMA_TWITTER_ZSCORED                  = transpose(V2_EMA_TWITTER_ZSCORED,                 feature_name  = 'EMA_TWITTER_ZSCORED')            
V2_EMA_REDDIT_ZSCORED                   = transpose(V2_EMA_REDDIT_ZSCORED,                  feature_name  = 'EMA_REDDIT_ZSCORED')            
V2_EMA_BLOOMBERG_ZSCORED                = transpose(V2_EMA_BLOOMBERG_ZSCORED,               feature_name  = 'EMA_BLOOMBERG_ZSCORED')                
V2_EMA_TWITTER_REDDIT_ZSCORED           = transpose(V2_EMA_TWITTER_REDDIT_ZSCORED,          feature_name  = 'EMA_TWITTER_REDDIT_ZSCORED')              
V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED = transpose(V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED,feature_name  = 'EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED')                
V2_OPEN_TO_CLOSE_STOCK_RETURNS          = transpose(V2_OPEN_TO_CLOSE_STOCK_RETURNS,         feature_name  = 'STOCK_RETURNS')
V2_OPEN_TO_CLOSE_EQUALW_RETURNS         = transpose(V2_OPEN_TO_CLOSE_EQUALW_RETURNS,        feature_name  = 'EQUAL_RETURNS')
V2_OPEN_TO_CLOSE_XLK_RETURNS            = transpose(V2_OPEN_TO_CLOSE_XLK_RETURNS,           feature_name  = 'XLK_RETURNS')        
V2_OPEN_TO_OPEN_STOCK_RETURNS           = transpose(V2_OPEN_TO_OPEN_STOCK_RETURNS,          feature_name  = 'STOCK_RETURNS')
V2_OPEN_TO_OPEN_EQUALW_RETURNS          = transpose(V2_OPEN_TO_OPEN_EQUALW_RETURNS,         feature_name  = 'EQUAL_RETURNS')
V2_OPEN_TO_OPEN_XLK_RETURNS             = transpose(V2_OPEN_TO_OPEN_XLK_RETURNS,            feature_name  = 'XLK_RETURNS')                   
                    

In [36]:
_list = [
                    V2_TWITTER_ZSCORED,
                    V2_REDDIT_ZSCORED,
                    V2_BLOOMBERG_ZSCORED,
                    V2_TWITTER_REDDIT_ZSCORED,
                    V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED,
                    V2_EMA_ZSCORED,
                    V2_EMA_TWITTER_ZSCORED,
                    V2_EMA_REDDIT_ZSCORED,
                    V2_EMA_BLOOMBERG_ZSCORED,
                    V2_EMA_TWITTER_REDDIT_ZSCORED,
                    V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED,
                    V2_OPEN_TO_CLOSE_STOCK_RETURNS,
                    V2_OPEN_TO_CLOSE_EQUALW_RETURNS,
                    V2_OPEN_TO_CLOSE_XLK_RETURNS,
                    V2_OPEN_TO_OPEN_STOCK_RETURNS ,
                    V2_OPEN_TO_OPEN_EQUALW_RETURNS,
                    V2_OPEN_TO_OPEN_XLK_RETURNS
    
    
    
                     ]

In [37]:
for i in _list:
    print(i.shape)

(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(16200, 3)
(216, 3)
(216, 3)
(16200, 3)
(216, 3)
(216, 3)


In [38]:
export_path = '/Users/niclaswolnerhanssen/Desktop/LINC/Sentiment LYNX/2_SIGNAL_PROCESSING/'

V2_TWITTER_ZSCORED.to_excel(
    export_path+'V2_TWITTER_ZSCORED.xlsx')
V2_REDDIT_ZSCORED.to_excel(
    export_path+'V2_REDDIT_ZSCORED.xlsx')
V2_BLOOMBERG_ZSCORED.to_excel(
    export_path+'V2_BLOOMBERG_ZSCORED.xlsx')
V2_TWITTER_REDDIT_ZSCORED.to_excel(
    export_path+'V2_TWITTER_REDDIT_ZSCORED.xlsx')
V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED.to_excel(
    export_path+'V2_TWITTER_REDDIT_BLOOMBERG_ZSCORED.xlsx')
V2_EMA_ZSCORED.to_excel(
    export_path+'V2_EMA_ZSCORED.xlsx')
V2_EMA_TWITTER_ZSCORED.to_excel(
    export_path+'V2_EMA_TWITTER_ZSCORED.xlsx')
V2_EMA_REDDIT_ZSCORED.to_excel(
    export_path+'V2_EMA_REDDIT_ZSCORED.xlsx')
V2_EMA_BLOOMBERG_ZSCORED.to_excel(
    export_path+'V2_EMA_BLOOMBERG_ZSCORED.xlsx')
V2_EMA_TWITTER_REDDIT_ZSCORED.to_excel(
    export_path+'V2_EMA_TWITTER_REDDIT_ZSCORED.xlsx')
V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED.to_excel(
    export_path+'V2_EMA_TWITTER_REDDIT_BLOOMBERG_ZSCORED.xlsx')
V2_OPEN_TO_CLOSE_STOCK_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_CLOSE_STOCK_RETURNS.xlsx')
V2_OPEN_TO_CLOSE_EQUALW_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_CLOSE_EQUALW_RETURNS.xlsx')
V2_OPEN_TO_CLOSE_XLK_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_CLOSE_XLK_RETURNS.xlsx')
V2_OPEN_TO_OPEN_STOCK_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_OPEN_STOCK_RETURNS.xlsx')
V2_OPEN_TO_OPEN_EQUALW_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_OPEN_EQUALW_RETURNS.xlsx')
V2_OPEN_TO_OPEN_XLK_RETURNS.to_excel(
    export_path+'V2_OPEN_TO_OPEN_XLK_RETURNS.xlsx')

